In [1]:
from dataParser import data_parsing 
import cupy 
import cudf
import nibabel as nib
import nilearn
import os

In [2]:
root = '/mnt/k/Abide Dataset/'

In [3]:
fmri_data_list = []
patients_ids = []
fmri_heads = []
patient_metadata = []

In [4]:
universities = os.listdir(root)
for university in universities[0:1]:
    directory = os.path.join(root, university)

    FolderList = os.listdir(directory)

    for i, folder in enumerate(FolderList):
        patients_ids.append(int(folder))
        rest_file = os.path.join(directory, folder, r'session_1/rest_1/rest.nii')
        if rest_file:
            fmri_img = nib.load(rest_file)
            fmri_data = fmri_img.get_fdata()
            fmri_header = fmri_img.header

            num_voxels = fmri_data.shape[0] * fmri_data.shape[1] * fmri_data.shape[2]
            seq_len = fmri_data.shape[3]

            fmri_data_model = fmri_data.reshape(num_voxels, seq_len).T
            fmri_data_list.append(fmri_data_model)
            fmri_heads.append(fmri_header)

        if i == 20:
            break

    
    # patient_metadata.append()
            
            
            

In [5]:
csv_path = os.path.join(root, "ABIDEII-EMC_1_CSV.csv")
patients_metadata = cudf.read_csv(csv_path)
patients_metadata = patients_metadata.fillna(0)

In [6]:
patients_metadata.head()

,SITE_ID,SUB_ID,NDAR_GUID,DX_GROUP,PDD_DSM_IV_TR,ASD_DSM_5,AGE_AT_SCAN,SEX,HANDEDNESS_CATEGORY,HANDEDNESS_SCORES,...,ADI_R_C3_TOTAL,ADI_R_C4_REPETITIVE_USE_OBJECTS,ADI_R_C4_HIGHER,ADI_R_C4_UNUSUAL_SENSORY_INTERESTS,ADI_R_C4_TOTAL,ADI_R_D_AGE_PARENT_NOTICED,ADI_R_D_AGE_FIRST_SINGLE_WORDS,ADI_R_D_AGE_FIRST_PHRASES,ADI_R_D_AGE_WHEN_ABNORMALITY,ADI_R_D_INTERVIEWER_JUDGMENT
0,ABIDEII-EMC_1,29864,0,1,0,0,9.013005,1,2,0,...,0,0,0,0,0,0,0,0,0,0
1,ABIDEII-EMC_1,29865,0,1,0,0,10.663929,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,ABIDEII-EMC_1,29866,0,1,0,0,8.720055,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,ABIDEII-EMC_1,29867,0,1,0,0,8.517454,2,1,0,...,0,0,0,0,0,0,0,0,0,0
4,ABIDEII-EMC_1,29868,0,1,0,0,10.650240,1,2,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
fmri_df = cudf.DataFrame(fmri_data_list)
fmri_df.shape

(21, 160)

In [10]:
patients_metadata.describe()

,SUB_ID,NDAR_GUID,DX_GROUP,PDD_DSM_IV_TR,ASD_DSM_5,AGE_AT_SCAN,SEX,HANDEDNESS_CATEGORY,HANDEDNESS_SCORES,FIQ,...,ADI_R_C3_TOTAL,ADI_R_C4_REPETITIVE_USE_OBJECTS,ADI_R_C4_HIGHER,ADI_R_C4_UNUSUAL_SENSORY_INTERESTS,ADI_R_C4_TOTAL,ADI_R_D_AGE_PARENT_NOTICED,ADI_R_D_AGE_FIRST_SINGLE_WORDS,ADI_R_D_AGE_FIRST_PHRASES,ADI_R_D_AGE_WHEN_ABNORMALITY,ADI_R_D_INTERVIEWER_JUDGMENT
count,54.000000,54.0,54.000000,54.0,54.0,54.000000,54.000000,54.000000,54.0,54.0,...,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0
mean,29890.500000,0.0,1.500000,0.0,0.0,8.098664,1.185185,1.203704,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,15.732133,0.0,0.504695,0.0,0.0,1.087599,0.392095,0.406533,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,29864.000000,0.0,1.000000,0.0,0.0,6.212183,1.000000,1.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,29877.250000,0.0,1.000000,0.0,0.0,7.199179,1.000000,1.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,29890.500000,0.0,1.500000,0.0,0.0,8.156057,1.000000,1.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,29903.750000,0.0,2.000000,0.0,0.0,8.826831,1.000000,1.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,29917.000000,0.0,2.000000,0.0,0.0,10.663929,2.000000,2.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
